In [1]:
import datetime
import pandas as pd
import time
import requests
import os
import numpy as np
import xlsxwriter
from abc import ABCMeta, abstractmethod
from Queue import Queue
from collections import deque
import getpass
import pandas.io.formats.excel

In [2]:
# TODOs:

# PNL reporting/collecting relevant statistics for visualizations from our simulations

# Longer time horizons. Only weight according to n most recent time steps

# Benchmarks: 
## - Buy and hold market index (Dow Jones)
## - Constantly Rebalanced Portfolio (maintain equal weights of each asset at all times)

# Non-fractional shares

# Contexts

# Other trading strategies
## Fundamentals-based strategies (need to get more data for this)



### Simulation runner

In [24]:

class SimulationEnv(object):
    
    loggables = ['positions']
    
    def __init__(self, wealth, assets, 
                 path_to_data, start_date, end_date,
                 agent_type, expert_type, reinvest, max_assets=100):

        self.init_wealth = wealth
        self.wealth = wealth
        self.assets = assets
        self.agent_type = agent_type
        self.expert_type = expert_type
        self.path_to_data = path_to_data
        self.reinvest = reinvest
        self.max_assets = max_assets
        ## TODO: 
        ## - specify start and end date for simulation
        ## - collect statistics on simulation results
        self.start_date = start_date
        self.end_date = end_date
        
        return
    
    
    ## To be called before running a simulation - initialize experts, agents, and initial position, etc.

    def setup_params(self, agent_args={}, expert_args={}):
    
        stdevs = {}
        for f in os.listdir(self.path_to_data):
            df = pd.read_csv(self.path_to_data+f)
            df = df[df["date"] < self.start_date]
            df["return"] = (df["adj_close"] - df["adj_close"].shift(1))/df["adj_close"].shift(1)
            stdevs[f] = df["return"].std()
            self.assets = [f.split(".")[0].upper() for f in sorted(stdevs, key=stdevs.get, reverse=True)[:self.max_assets]]
        print self.assets
        self.experts = [self.expert_type(a, self.path_to_data, **expert_args) for a in self.assets]
        self.agent = self.agent_type(self.experts, **agent_args)
        self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
        
        ## TODO: 
        # do various other stuff here like for select for high-volatility stocks or something
        # exception handling
        # Need to make sure data files are in sync
        
        return
        
    ## Run simulation
    def run(self, log=False, logpath=os.getcwd()):
        
        ## Start period counter and log
        self.period = 1
        self.finallog = []
        self.log = log
        self.logpath = logpath
        
        ## Warmup period: 
        ## i.e. for strategies involving moving average indicators, wait until we have enough data to calculate MA
        for expert in self.agent.experts:
            expert.warmup()
        
        ## Simulation: Go until data iterators reach the end
        while True:
            try:
                print "PERIOD {}".format(self.period)
                print "positions:"
                print np.array([weight * self.wealth for weight in self.agent.weights])
                print "weights:"
                print np.array([weight for weight in self.agent.weights])
                
                if self.period > 1:
                    print "rewards:"
                    print np.array([r for r in self.agent.rewards])
                    print "returns:"
                    print np.array([r for r in self.agent.returns])

                print "---------------------"
                ## Log this period
                if log:
                    self.logperiod()
                    
                ## Update experts with last period's rewards
                for i, expert in enumerate(self.agent.experts):
                    expert.update()
                    if self.reinvest:# below is to redistribute unallocated money
                        if expert.pick == False:
                            extra_money = self.positions[i]
                            self.positions[i] = 0
                            distribute = np.greater(self.positions, 0)
                            if not np.any(distribute): # this is to avoid divide by zero if nothing is invested
                                #print("all experts say not to buy")
                                self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
                            else:
                                distribute_extra_money = distribute * (extra_money/ np.sum(distribute))
                                self.positions = np.add(self.positions, distribute_extra_money)
                            assert(abs(np.sum(self.positions)-self.wealth) <= 1)# make sure all money is accounted for
                      
                ## Update agent accordingly (i.e. for Hedge, update weights according to each expert's reward in the last period)
                self.agent.update()
                

                
                ## Update position with returns from last period
                self.positions = self.positions * (1 + self.agent.returns)
                self.wealth = np.sum(self.positions)
                
                ## Rebalance according to new, updated weights
                ## TODO: only allow non-fractional shares to be purchased (?)
                if np.sum(self.agent.weights) == 1:
                    self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
                    
                print "new weights:"
                print np.array([weight for weight in self.agent.weights])
                print "---------------------"
                print "\n\n"
                
                
                ## Advance period
                self.period += 1
                
            except StopIteration:
                break
        
        ## Write the log file
        if log:
            self.savelog()

    def logperiod(self):
        row = [self.period] + [self.wealth]
        nrow = []
        for loggable in (self.loggables):
            if getattr(self,loggable) is None:
                nrow += [None] * len(self.experts)
            else:
                nrow += getattr(self,loggable).tolist()
        arow = []
        for loggable in (self.agent_type.loggables):
            if getattr(self.agent,loggable) is None:
                arow += [None] * len(self.experts)
            else:
                arow += getattr(self.agent,loggable).tolist()
        erow = []
        for loggable in (self.expert_type.loggables):
            if [getattr(e,loggable) for e in self.experts] is None:
                erow += [None] * len(self.experts)
            else:
                erow += [getattr(e,loggable) for e in self.experts]
        row += nrow + arow + erow
        self.finallog.append(row)
    
    def savelog(self):
        runtime = datetime.datetime.now()
        runuser = getpass.getuser()
        logname = runuser + "_" + runtime.strftime('%Y-%m-%d_%H-%M-%S')
        if not os.path.exists(os.path.join(self.logpath, logname)):
            os.makedirs(os.path.join(self.logpath, logname))
        col = ['period', 'wealth'] + \
            [x+'.'+y for x in self.loggables for y in self.assets] + \
            [x+'.'+y for x in self.agent_type.loggables for y in self.assets] + \
            [x+'.'+y for x in self.expert_type.loggables for y in self.assets]
        df = pd.DataFrame(self.finallog, columns=col)
        df.set_index('period', inplace=True)
        
        
        writer = pd.ExcelWriter(os.path.join(self.logpath, logname, logname+'.xlsx'), engine='xlsxwriter')
        pd.io.formats.excel.header_style = None
        df.to_excel(writer,'run_log')
        
        workbook  = writer.book
        worksheet = writer.sheets['run_log']
        
        header_format = workbook.add_format({'bold': True,'text_wrap': True})
        worksheet.set_row(0, None, header_format)
        writer.save()



### Base classes

Base classes for agents and experts to be implemented by us

In [25]:

class Agent(object):
    __metaclass__ =ABCMeta
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
    @abstractmethod
    def act(self):
        pass
    
class Expert(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, reward_data):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
class Context(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, context_data):
        pass
    
    @abstractmethod
    def observe(self):
        pass

### Agents

Implementations of different online portfolio selection algorithms
* Exponential Gradient
* Exponential Gradient (recent history only)

In [26]:
## EG. (http://www.cis.upenn.edu/~mkearns/finread/helmbold98line.pdf)
class EG(Agent):
    
    loggables = ['returns','rewards','weights']
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta):
        self.eta = eta
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = np.asarray([e.returns for e in self.experts])
        multipliers = np.exp(self.eta * self.rewards/np.sum(self.weights * self.rewards))
        self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
        self.weights = np.nan_to_num(self.weights)

        return
    
    def act(self):
        return self.weights


class EGRecent(Agent):
    
    loggables = ['returns','rewards','weights']
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta, n):
        self.eta = eta
        self.experts = experts
        #self.weights = Queue(maxsize=n)
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = deque(maxlen=n)
        self.returns = None
        self.t = 0
        self.n = n
        #self.rewards = None
        #self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
           
        self.rewards.append(np.asarray([e.reward for e in self.experts]))
        self.returns = np.asarray([e.returns for e in self.experts])
        
        self.weights = np.ones(len(self.experts))/len(self.experts)
        #print self.t
        #print len(self.rewards)
        for rewards in self.rewards:
            rewards = np.asarray(rewards)
            multipliers = np.exp(self.eta * rewards/np.sum(self.weights * rewards))
            self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
            self.weights = np.nan_to_num(self.weights)
        self.t += 1
        #print self.t
        
        return
    
    def act(self):
        return self.weights
    
      
        

More agents to serve as naive/benchmark portfolio allocation algorithms:
* Equal-weighted buy and hold
* Market-weighted buy and hold
* Constantly rebalanced portfolio


In [27]:
## Some other examples of agents to use as benchmarks 

class NaiveBuyHold(Agent):
    
    loggables = ['rewards','weights']
    
    def __init__(self, experts):
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        return
    
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = self.rewards
        return
    
    def act(self):
        return self.weights
    
## TODO
class WeightedBuyHold(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    

## to be used with "Dummy" Expert for each stock
class ConstantRebalancer(Agent):
    
    def __init__(self, experts):
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        return
    
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = self.rewards
        multipliers = np.asarray([e.reward + 1 for e in self.experts])
        self.weights = np.multiply(self.weights, np.reciprocal(multipliers))
        self.weights = np.divide(self.weights, np.sum(self.weights))
        assert((np.sum(self.weights) - 1) <= 0.00001)
        return
    
    def act(self):
        return self.weights

### Experts

* Dummy
* Mean Reversion
* Momentum

In [28]:

## Dummy expert that always pick the same asset
class Dummy(Expert):
    
    loggables = ['reward']
    
    ## Expert has a reward associated with its pick
    def __init__(self, name, path_to_data):
        self.reward = 0.
        self.pick = True ## Might not be necessary
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        return
    
    ## Expert updates its reward 
    def update(self):
        current_price = float(self.data.get_chunk(1)["adj_close"])
        self.reward = current_price/self.last_price
        self.returns = self.reward - 1.
        self.last_price = current_price
        return
    
    def warmup(self):
        pass
    

class MeanReversion(Expert):
    
    loggables = ['avg','std']
    
    def __init__(self, name, path_to_data, window_size, threshold):
        self.reward = 0.
        self.pick = False
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        self.window_size = window_size
        self.avg = 0.0
        self.std = 0.0
        self.threshold = threshold
        self.last_n_prices = Queue(maxsize=10)
        self.returns = 0.
        return
    
    def warmup(self):
        n = 1
        while n <= self.window_size:
            self.last_n_prices.put(self.last_price)
            self.last_price = float(self.data.get_chunk(1)["adj_close"])
            n += 1
        return
        
    def update(self):
        _ = self.last_n_prices.get()
            
        self.last_n_prices.put(self.last_price)
        self.avg = np.mean(list(self.last_n_prices.queue))
        self.std = np.std(list(self.last_n_prices.queue))
        
        current_price = float(self.data.get_chunk(1)["adj_close"])

        ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
        if self.pick:
            self.reward = current_price/self.last_price
            self.returns = self.reward - 1.
        else:
            self.reward = -current_price/self.last_price
            self.returns = 0
        self.last_price = current_price

        if self.last_price <= self.avg - self.threshold * self.std:
            self.pick = True
        else:
            self.pick = False

        return
    
    
class Momentum(Expert):
    
    loggables = ['avg','std']
    
    def __init__(self, name, path_to_data, window_size, threshold):
        self.reward = 0.
        self.pick = False
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.last_price = float(self.data.get_chunk(1)["adj_close"])
        self.window_size = window_size
        self.avg = 0.0
        self.std = 0.0
        self.threshold = threshold
        self.last_n_prices = Queue(maxsize=10)
        self.returns = 0.
        return
    
    def warmup(self):
        n = 1
        while n <= self.window_size:
            self.last_n_prices.put(self.last_price)
            self.last_price = float(self.data.get_chunk(1)["adj_close"])
            n += 1
        return
        
    def update(self):
        _ = self.last_n_prices.get()
            
        self.last_n_prices.put(self.last_price)
        self.avg = np.mean(list(self.last_n_prices.queue))
        self.std = np.std(list(self.last_n_prices.queue))
        
        current_price = float(self.data.get_chunk(1)["adj_close"])

        ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
        if self.pick:
            self.reward = current_price/self.last_price
            self.returns = self.reward - 1.
        else:
            self.reward = -current_price/self.last_price
            self.returns = 0
        self.last_price = current_price

        if self.last_price >= self.avg - self.threshold * self.std:
            self.pick = True
        else:
            self.pick = False

        return
  

### Contexts

In [29]:
  
class VolContext(Context):
    
    def __init__(self, data_file, threshold):
        self.data = pd.read_csv(data_file, iterator=True, chunksize=1)
        self.threshold = threshold
        self.contexts = ["HighVol", "LowVol"]
        return
    
    # Returns a string giving the name of the context
    def observe(self):
        if float(self.data.get_chunk(1)["adj_close"]) > self.threshold:
            return self.contexts[0]
        else:
            return self.contexts[1]

In [30]:
stocks = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GE',
 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 
'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'VZ', 'WMT', 'XOM']
stocks = [stock.lower() for stock in stocks]

In [31]:
initial_wealth = 100000

In [33]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2016-01-01", "2017-11-01", EG, Dummy, True, max_assets=5)
s.setup_params(agent_args={"eta": 0.01})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

['MSFT', 'CVX', 'DD', 'AAPL', 'CAT']
PERIOD 1
positions:
[ 20000.  20000.  20000.  20000.  20000.]
weights:
[ 0.2  0.2  0.2  0.2  0.2]
---------------------
new weights:
[ 0.20004573  0.19998239  0.20000918  0.20000676  0.19995594]
---------------------



PERIOD 2
positions:
[ 19375.50921855  19369.37503682  19371.9697586   19371.73496985
  19366.81249853]
weights:
[ 0.20004573  0.19998239  0.20000918  0.20000676  0.19995594]
rewards:
[ 0.99069718  0.96002842  0.97300231  0.97182841  0.94721376]
returns:
[-0.00930282 -0.03997158 -0.02699769 -0.02817159 -0.05278624]
---------------------
new weights:
[ 0.20002966  0.19999473  0.19999915  0.20002022  0.19995625]
---------------------



PERIOD 3
positions:
[ 19245.36022854  19241.99979339  19242.42528522  19244.45205883
  19238.29755053]
weights:
[ 0.20002966  0.19999473  0.19999915  0.20002022  0.19995625]
rewards:
[ 0.98542903  0.99953738  0.98842722  1.00009412  0.99356544]
returns:
[ -1.45709660e-02  -4.62620281e-04  -1.15727830e-02

---------------------
new weights:
[ 0.19984163  0.19997558  0.20009354  0.20022955  0.19985971]
---------------------



PERIOD 24
positions:
[ 19690.02259554  19703.22070265  19714.84270173  19728.24333527
  19691.80388188]
weights:
[ 0.19984163  0.19997558  0.20009354  0.20022955  0.19985971]
rewards:
[ 1.01457935  1.00886018  1.03064822  1.0071094   1.01976815]
returns:
[ 0.01457935  0.00886018  0.03064822  0.0071094   0.01976815]
---------------------
new weights:
[ 0.19984359  0.19998493  0.20010003  0.20021651  0.19985493]
---------------------



PERIOD 25
positions:
[ 19652.58317891  19666.48263296  19677.80128173  19689.25587235
  19653.69835104]
weights:
[ 0.19984359  0.19998493  0.20010003  0.20021651  0.19985493]
rewards:
[ 0.99905771  1.00274449  1.00131579  0.99157912  0.99569223]
returns:
[-0.00094229  0.00274449  0.00131579 -0.00842088 -0.00430777]
---------------------
new weights:
[ 0.19983111  0.19998967  0.20007994  0.20021959  0.19987969]
---------------------





new weights:
[ 0.1998577   0.19987794  0.20015585  0.20033169  0.19977682]
---------------------



PERIOD 45
positions:
[ 19862.85609287  19864.86741062  19892.48762193  19909.96371447
  19854.81742939]
weights:
[ 0.1998577   0.19987794  0.20015585  0.20033169  0.19977682]
rewards:
[ 1.01156752  1.00386287  1.00550294  1.0042654   1.00936797]
returns:
[ 0.01156752  0.00386287  0.00550294  0.0042654   0.00936797]
---------------------
new weights:
[ 0.19984263  0.19988133  0.20018467  0.20031345  0.19977792]
---------------------



PERIOD 46
positions:
[ 19629.30172902  19633.10334664  19662.89808198  19675.54699641
  19622.94577532]
weights:
[ 0.19984263  0.19988133  0.20018467  0.20031345  0.19977792]
rewards:
[ 0.98086348  0.98999519  1.0025455   0.97931414  0.98886276]
returns:
[-0.01913652 -0.01000481  0.0025455  -0.02068586 -0.01113724]
---------------------
new weights:
[ 0.19984399  0.19989734  0.20020166  0.20028061  0.1997764 ]
---------------------



PERIOD 47
positions:
[

new weights:
[ 0.19983628  0.19995675  0.20001122  0.20035742  0.19983833]
---------------------



PERIOD 69
positions:
[ 19583.89670067  19595.70241945  19601.04095944  19634.96781074
  19584.0968472 ]
weights:
[ 0.19983628  0.19995675  0.20001122  0.20035742  0.19983833]
rewards:
[ 1.00095877  0.996165    0.99626194  0.99803304  0.99443099]
returns:
[ 0.00095877 -0.003835   -0.00373806 -0.00196696 -0.00556901]
---------------------
new weights:
[ 0.19981928  0.19998865  0.20000773  0.20033698  0.19984735]
---------------------



PERIOD 70
positions:
[ 19698.24436458  19714.94089973  19716.8221329   19749.27956889
  19701.01193427]
weights:
[ 0.19981928  0.19998865  0.20000773  0.20033698  0.19984735]
rewards:
[ 0.9973659   1.02197183  1.00416898  0.99566417  1.01046993]
returns:
[-0.0026341   0.02197183  0.00416898 -0.00433583  0.01046993]
---------------------
new weights:
[ 0.19982564  0.19999482  0.19998099  0.20032211  0.19987644]
---------------------



PERIOD 71
positions:
[

new weights:
[ 0.20002917  0.19990201  0.19987723  0.20029261  0.19989897]
---------------------



PERIOD 93
positions:
[ 20665.80094684  20652.6633413   20650.10302179  20693.01801709
  20652.34965203]
weights:
[ 0.20002917  0.19990201  0.19987723  0.20029261  0.19989897]
rewards:
[ 0.99127668  1.00599355  1.00832496  0.99860411  0.99650665]
returns:
[-0.00872332  0.00599355  0.00832496 -0.00139589 -0.00349335]
---------------------
new weights:
[ 0.20002251  0.19988158  0.19987498  0.20031994  0.19990097]
---------------------



PERIOD 94
positions:
[ 20611.76532196  20597.24270694  20596.56276895  20642.41445504
  20599.24096898]
weights:
[ 0.20002251  0.19988158  0.19987498  0.20031994  0.19990097]
rewards:
[ 0.99409877  0.98722577  0.99629893  1.01102741  0.99841683]
returns:
[-0.00590123 -0.01277423 -0.00370107  0.01102741 -0.00158317]
---------------------
new weights:
[ 0.20002255  0.199856    0.19991611  0.20032315  0.19988219]
---------------------



PERIOD 95
positions:
[

...

rewards:
[ 0.99536393  0.98180155  0.99002831  0.99849138  0.99158031]
returns:
[-0.00463607 -0.01819845 -0.00997169 -0.00150862 -0.00841969]
---------------------
new weights:
[ 0.20034918  0.19970359  0.19999574  0.20017245  0.19977903]
---------------------



PERIOD 554
positions:
[ 25522.98759498  25440.7437193   25477.9620947   25500.47319982
  25450.35443527]
weights:
[ 0.20034918  0.19970359  0.19999574  0.20017245  0.19977903]
rewards:
[ 1.00527868  1.01415797  1.01330515  1.0105763   1.0163292 ]
returns:
[ 0.00527868  0.01415797  0.01330515  0.0105763   0.0163292 ]
---------------------
new weights:
[ 0.2003561   0.19969509  0.19998436  0.20018608  0.19977837]
---------------------



PERIOD 555
positions:
[ 25392.56350638  25308.7888101   25345.44963635  25371.0149734
  25319.34357971]
weights:
[ 0.2003561   0.19969509  0.19998436  0.20018608  0.19977837]
rewards:
[ 0.99830116  0.99063189  0.98920113  1.00163748  0.99453728]
returns:
[-0.00169884 -0.00936811 -0.01079887  0.0

[-0.00137447 -0.00461524 -0.0026892  -0.00444194  0.00265562]
---------------------
new weights:
[ 0.20039099  0.19964972  0.19993576  0.2002084   0.19981514]
---------------------



PERIOD 578
positions:
[ 25116.82167825  25023.91139994  25059.76352453  25093.93601218
  25044.64507076]
weights:
[ 0.20039099  0.19964972  0.19993576  0.2002084   0.19981514]
rewards:
[ 0.9946475   0.98627933  0.99858757  0.99631728  0.9902532 ]
returns:
[-0.0053525  -0.01372067 -0.00141243 -0.00368272 -0.0097468 ]
---------------------
new weights:
[ 0.20038381  0.19964091  0.19992783  0.200212    0.19983545]
---------------------



PERIOD 579
positions:
[ 25384.40577981  25290.29639294  25326.64297871  25362.64091296
  25314.94071864]
weights:
[ 0.20038381  0.19964091  0.19992783  0.200212    0.19983545]
rewards:
[ 1.00707257  1.00623621  1.00668638  1.01251066  1.02096929]
returns:
[ 0.00707257  0.00623621  0.00668638  0.01251066  0.02096929]
---------------------
new weights:
[ 0.2004058   0.1996356

positions:
[ 26662.64925964  26555.90804679  26585.86553502  26647.24966705
  26599.94351505]
weights:
[ 0.20039328  0.19959102  0.19981618  0.20027753  0.19992199]
rewards:
[ 1.00336012  1.00141349  0.99948307  0.99876615  1.01310001]
returns:
[ 0.00336012  0.00141349 -0.00051693 -0.00123385  0.01310001]
---------------------
new weights:
[ 0.20039015  0.19958437  0.19983725  0.20026582  0.19992241]
---------------------



PERIOD 603
positions:
[ 26738.28220242  26630.76528179  26664.50738794  26721.69280948
  26675.87137906]
weights:
[ 0.20039015  0.19958437  0.19983725  0.20026582  0.19992241]
rewards:
[ 1.00131043  0.9995295   1.01344712  0.9970091   1.00308791]
returns:
[ 0.00131043 -0.0004705   0.01344712 -0.0029909   0.00308791]
---------------------
new weights:
[ 0.20041367  0.19956229  0.19981079  0.2002715   0.19994175]
---------------------



PERIOD 604
positions:
[ 26757.92360824  26644.25319642  26677.43097298  26738.94264456
  26694.91558845]
weights:
[ 0.20041367  0.1

  26440.43557225]
weights:
[ 0.20042259  0.19956376  0.19991463  0.20015741  0.1999416 ]
rewards:
[ 0.98128456  0.99932783  0.99420897  0.98573584  0.99693046]
returns:
[-0.01871544 -0.00067217 -0.00579103 -0.01426416 -0.00306954]
---------------------
new weights:
[ 0.20041703  0.19954538  0.19990668  0.20016376  0.19996714]
---------------------



PERIOD 627
positions:
[ 26803.62454598  26687.0510313   26735.37072247  26769.75264253
  26743.45611634]
weights:
[ 0.20041703  0.19954538  0.19990668  0.20016376  0.19996714]
rewards:
[ 1.00852478  1.00201787  1.00731193  1.01454014  1.02424709]
returns:
[ 0.00852478  0.00201787  0.00731193  0.01454014  0.02424709]
---------------------
new weights:
[ 0.2004004   0.19956358  0.19990763  0.20015529  0.1999731 ]
---------------------



PERIOD 628
positions:
[ 26518.45799522  26407.72387083  26453.25069034  26486.02379702
  26461.913847  ]
weights:
[ 0.2004004   0.19956358  0.19990763  0.20015529  0.1999731 ]
rewards:
[ 0.98123209  0.998465

positions:
[ 28018.14205386  27902.36763957  27941.89566158  27979.80605499
  27963.47092342]
weights:
[ 0.20040775  0.19957964  0.19986237  0.20013354  0.2000167 ]
rewards:
[ 0.99807481  1.01456177  1.00547379  1.00806181  0.99254059]
returns:
[-0.00192519  0.01456177  0.00547379  0.00806181 -0.00745941]
---------------------
new weights:
[ 0.20038473  0.19955973  0.19984001  0.20021322  0.20000231]
---------------------



PERIOD 651
positions:
[ 28215.41688147  28099.25146306  28138.71625372  28191.26733154
  28161.5688653 ]
weights:
[ 0.20038473  0.19955973  0.19984001  0.20021322  0.20000231]
rewards:
[ 0.99559107  0.99711139  0.99588677  1.04725092  0.99991158]
returns:
[ -4.40892808e-03  -2.88860805e-03  -4.11323494e-03   4.72509164e-02
  -8.84173325e-05]
---------------------
new weights:
[ 0.20039323  0.19955253  0.19982737  0.20020466  0.20002221]
---------------------



PERIOD 652
positions:
[ 28054.69702476  27937.00089232  27975.47764869  28028.29757036
  28002.75493744]


In [ ]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", NaiveBuyHold, Dummy, True)
s.setup_params(agent_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

In [ ]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", ConstantRebalancer, Dummy, True)
s.setup_params(agent_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

In [ ]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", ConstantRebalancer, Dummy, True)
s.setup_params(expert_args={"eta": -0.01})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01})
s.run(log=True, logpath="logs")

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EGRecent, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01, "n": 100})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01, "n": 750})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", NaiveBuyHold, Dummy, True)
s.setup_params(expert_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))



In [ ]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", NaiveBuyHold, Dummy, True)
s.setup_params(expert_args={})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run simulation where we each expert only recommends to buy when the price crosses 0.5 standard deviations below the 10-day moving average. 
## Otherwise doesn't do anything
## Sell position when price gets above 0.5 standard deviations below the 10-day MA

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, MeanReversion, True)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, MeanReversion, True)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, MeanReversion, True)

s.setup_params(
    agent_args={"eta": -0.01, "n": 1250},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, Momentum, True)

years = 2. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, Momentum, True)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Momentum, True)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.001, "n": 500},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
pd.__version__